## KEEN paper

This is an attempt to reproduce essential bits of the [KEEN](https://arxiv.org/abs/2406.12673) paper.

## Dataset exploration
1. Figure out how to produce (input-label) pairs followed by hidden states of entities in the `input`.

__Question__
- How to generate labels?
2. Create prompt and run through the model to generate hidden states across some layers.

__Question__
- How best to use min-max scaler on hidden state per layer without loading from huggingface.
- Will using models layernorm improve the result?

## Facts from PopQA dataset exploration
- Number of rows in dataset: 14267
- Number of unique subject name: 10415
- Number of unique subject ids: 12244
- Unique facts in the dataset by looking  at number of unique (subj, s_uri, prop): 13797
- Number of unique questions: 13068

## Insights from PopQA dataset exploration
- Since number of subject names is less than number of subject ids, this suggests that each subject name can potentially have multiple subject ids.
- Each subject name can have multiple facts associated to it.
- Each subject name can have multiple questions associated to it.


In [ ]:
# @title Install dependencies
!pip -q install datasets huggingface_hub

In [ ]:
from datasets import load_dataset

# Load POPQA dataset from huggingface
dataset = load_dataset("akariasai/PopQA")

# Let's list out the columns in the dataset
print(dataset)
print(dataset["test"].column_names)

README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


test.tsv: 0.00B [00:00, ?B/s]

Generating test split:   0%|          | 0/14267 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['id', 'subj', 'prop', 'obj', 'subj_id', 'prop_id', 'obj_id', 's_aliases', 'o_aliases', 's_uri', 'o_uri', 's_wiki_title', 'o_wiki_title', 's_pop', 'o_pop', 'question', 'possible_answers'],
        num_rows: 14267
    })
})
['id', 'subj', 'prop', 'obj', 'subj_id', 'prop_id', 'obj_id', 's_aliases', 'o_aliases', 's_uri', 'o_uri', 's_wiki_title', 'o_wiki_title', 's_pop', 'o_pop', 'question', 'possible_answers']


In [ ]:
# Please extract out subj and subj_id from the dataset and compare the lengths
subj = dataset["test"]["subj"]
subj_ids = dataset["test"]["subj_id"]

In [ ]:
# Get length of subj and get length of unique subjects in the dataset
print("Length of subject", len(subj))
print("Length of unique subject", len(set(subj)))

Length of subject 14267
Length of unique subject 10415


In [ ]:
# Get length of subj id and unique length of subj id
print("Length of unique subject names", len(set(subj)))
print("Length of unique subject ids", len(set(subj_ids)))

Length of unique subject names 10415
Length of unique subject ids 12244


Since length of subject ids is less than name of subject, this suggests that each subject name might have multiple subject ids.

In [ ]:
# Lets count the number of rows with unique ("subject", "s_uri", "prop")
# This is essential for counting unique (subject, relation, object) fact in the dataset
import pandas as pd

df = pd.DataFrame(dataset["test"])
df = df.groupby(["subj", "s_uri", "prop"]).size().reset_index(name="count")
print("Number of unique facts in the dataset", len(df))

Number of unique facts in the dataset 13797


In [ ]:
# Get number of unique questions in the dataset. Ensure to strip white spaces in the question
questions = dataset["test"]["question"]
questions = [q.strip() for q in questions]
print("Number of unique questions in the dataset", len(set(questions)))

Number of unique questions in the dataset 13068


In [ ]:
# @title Explore KEEN subject dataset

subj_dataset_train = load_dataset("dhgottesman/keen_estimating_knowledge_in_llms", data_files="popqa_train_subjects.csv")
subj_dataset_test = load_dataset("dhgottesman/keen_estimating_knowledge_in_llms", data_files="popqa_test_subjects.csv")
subj_dataset_val = load_dataset ("dhgottesman/keen_estimating_knowledge_in_llms", data_files="popqa_val_subjects.csv")

In [ ]:
subj_dataset_train

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'subject'],
        num_rows: 2224
    })
})

In [ ]:
# Get total length of train, test and val
subj_train_length = len(subj_dataset_train["train"])
subj_test_length = len(subj_dataset_test["train"])
subj_val_length = len(subj_dataset_val["train"])
print(subj_train_length + subj_test_length + subj_val_length)

3475


In [ ]:
# @title Explore KEEN PopQA questions
from datasets import load_dataset

keen_popqa_dataset = load_dataset("dhgottesman/keen_estimating_knowledge_in_llms", data_files="popqa_questions.csv")
keen_popqa_dataset

popqa_questions.csv: 0.00B [00:00, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'subj', 's_uri', 'o_uri', 'prop', 'obj', 'question', 's_aliases', 'o_aliases', 'possible_answers', 'label'],
        num_rows: 20896
    })
})

In [ ]:
# Convert keen_popqa_dataset from huggingface dataset to pandas dataframe
keen_popqa_df = keen_popqa_dataset["train"].to_pandas()
keen_popqa_df.head()

,Unnamed: 0,subj,s_uri,o_uri,prop,obj,question,s_aliases,o_aliases,possible_answers,label
0,0,'71,http://www.wikidata.org/entity/Q12100227,http://www.wikidata.org/entity/Q1174756,composer,David Holmes,Who was the composer of '71?,"['Seventy One', 'Seventy-one']",[],['David Holmes'],head
1,1,'71,http://www.wikidata.org/entity/Q12100227,http://www.wikidata.org/entity/Q130232,genre,drama film,What genre is '71?,"['Seventy One', 'Seventy-one']",['drama movie'],"['crime film', 'thriller movie', 'film action'...",head
2,2,'71,http://www.wikidata.org/entity/Q12100227,http://www.wikidata.org/entity/Q145,country of origin,United Kingdom,What is the country of origin of '71?,"['Seventy One', 'Seventy-one']","['GBR', 'The UK', 'The United Kingdom of Great...","['GBR', 'The UK', 'The United Kingdom of Great...",head
3,3,'71,http://www.wikidata.org/entity/Q12100227,http://www.wikidata.org/entity/Q15715283,director,Yann Demange,Who was the director of '71?,"['Seventy One', 'Seventy-one']",[],['Yann Demange'],head
4,4,'71,http://www.wikidata.org/entity/Q12100227,http://www.wikidata.org/entity/Q22006653,color,color,What color is '71?,"['Seventy One', 'Seventy-one']","['color film', 'colour', 'full color', 'colour...","['colour film', 'color film', 'color', 'colour...",head


In [ ]:
# @title Deterministic Run
import random
import numpy as np
import torch
import os

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
torch.set_num_threads(1)
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [ ]:
# @title Answer generation script


import torch
import pandas as pd

from tqdm import tqdm
torch.set_grad_enabled(False)
tqdm.pandas()

# from utils import set_hs_patch_hooks, remove_hooks

def set_hs_patch_hooks(model, hs_patch_config):
    def patch_hs(name, position_hs):
        def hook(module, input, output):
            for position_, hs_ in position_hs:
                # (batch, sequence, hidden_state)
                output[0][0, position_] = hs_

        return hook

    hooks = []
    for layer in hs_patch_config:
        hooks.append(model.model.layers[layer].register_forward_hook(
            patch_hs(f"patch_hs_{layer}", hs_patch_config[layer])
        ))

    return hooks

def remove_hooks(hooks):
    for hook in hooks:
        hook.remove()


def generate_greedy_deterministic(model, tokenizer, hs_patch_config, inp, max_length, end_token):
    input_ids = inp["input_ids"].detach().clone().cuda()
    with torch.no_grad():
        for _ in range(max_length):
            if hs_patch_config is None:
                outputs = model(input_ids, output_attentions=True, output_hidden_states=True)
            else:
                patch_hooks = set_hs_patch_hooks(model, hs_patch_config)
                outputs = model(input_ids, output_attentions=True, output_hidden_states=True)
                remove_hooks(patch_hooks)

            logits = outputs.logits[:, -1, :]
            next_token_id = torch.argmax(logits, dim=-1)
            input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0)], dim=-1)

            if next_token_id.item() == end_token:
                break
    generated_text = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
    return "".join(generated_text)

def decode_tokens(tokenizer, token_array):
  if hasattr(token_array, "shape") and len(token_array.shape) > 1:
    return [decode_tokens(tokenizer, row) for row in token_array]
  return [tokenizer.decode([t]) for t in token_array]

def find_token_range(tokenizer, token_array, substring):
  """Find the tokens corresponding to the given substring in token_array."""
  toks = decode_tokens(tokenizer, token_array)
  whole_string = "".join(toks)
  char_loc = whole_string.index(substring)
  loc = 0
  tok_start, tok_end = None, None
  for i, t in enumerate(toks):
    loc += len(t)
    if tok_start is None and loc > char_loc:
      tok_start = i
    if tok_end is None and loc >= char_loc + len(substring):
      tok_end = i + 1
      break
  return (tok_start, tok_end)

def generate(model, tokenizer, device, df, hs_patch_config=None):
    df = df[["question"]].drop_duplicates().reset_index().drop("index", axis=1)
    records = []
    for _, row in tqdm(df.iterrows()):
        inp = tokenizer(row["question"], return_tensors="pt").to(device)
        deterministic_generation = generate_greedy_deterministic(model, tokenizer, hs_patch_config, inp, 64, tokenizer.eos_token_id)
        record = {
            "question": row["question"],
            "deterministic_generation": deterministic_generation,
        }
        records.append(record)
    df = pd.DataFrame(records)
    return df

## What I need to generate question and answer
- Generate question-answer pair -> question-label pair
- Generate hidden state per answer

In [ ]:
# @title Set up model and tokenizer
import torch
import transformers
import re

def set_requires_grad(requires_grad, *models):
  for model in models:
    if isinstance(model, torch.nn.Module):
      for param in model.parameters():
        param.requires_grad = requires_grad
    elif isinstance(model, (torch.nn.Parameter, torch.Tensor)):
      model.requires_grad = requires_grad
    else:
      assert False, "unknown type %r" % type(model)

class GPTModelAndTokenizer:
  """An object to hold a GPT-style language model and tokenizer."""

  def __init__(
      self,
      model_name=None,
      model=None,
      tokenizer=None,
      low_cpu_mem_usage=False,
      torch_dtype=None,
      ):
    if tokenizer is None:
      assert model_name is not None
      tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    if model is None:
      assert model_name is not None
      model = transformers.AutoModelForCausalLM.from_pretrained(
          model_name, low_cpu_mem_usage=low_cpu_mem_usage,
          torch_dtype=torch_dtype
          )
      set_requires_grad(False, model)
    self.tokenizer = tokenizer
    self.model = model
    self.layer_names = [
        n
        for n, _ in model.named_modules()
        if (re.match(r"^(transformer|gpt_neox)\.(h|layers)\.\d+$", n))
    ]
    self.num_layers = len(self.layer_names)
    self.vocabulary_projection_function = lambda x, layer: self.model.lm_head(self.model.transformer.ln_f(x)) if layer < self.num_layers else self.model.lm_head(x)
    self.mlp_hidden_size = self.model.config.n_embd * 4
    print(self.mlp_hidden_size)
    print(self.model.config)


In [ ]:
# Start with gpt2 for testing that code works
# Subsequently used gpt2-xl

gpt_model = GPTModelAndTokenizer(model_name="gpt2-xl")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

6400
GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "dtype": "float32",
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.57.3",
  "use_cache": true,
  "vocab_size": 50257
}



In [ ]:
# @title Generate sample answers for questions on original PopQA

# Grab random 5 questions from the dataset and generate deterministic answer and display

sample = dataset["test"].select(range(5))
sample_df = sample.to_pandas()

# Grab question and possible_answers field and display
sample_df = sample_df[["question", "possible_answers"]]
sample_df.head()

,question,possible_answers
0,What is George Rankin's occupation?,"[""politician"", ""political leader"", ""political ..."
1,What is John Mayne's occupation?,"[""journalist"", ""journo"", ""journalists""]"
2,What is Henry Feilden's occupation?,"[""politician"", ""political leader"", ""political ..."
3,What is Kathy Saltzman's occupation?,"[""politician"", ""political leader"", ""political ..."
4,What is Eleanor Davis's occupation?,"[""cartoonist"", ""graphic artist"", ""animator"", ""..."


In [ ]:
# Let' use generate function to generate answer to a question

# Check for whether is it cuda or cpu
# params to generate function(model, tokenizer, device, df, hs_patch_config=None)
device = "cuda" if torch.cuda.is_available() else "cpu"
gpt_model.model.to(device) # Move the model to the correct device
answer_df = generate(gpt_model.model, gpt_model.tokenizer, device, sample_df)
answer_df.head()

5it [00:24,  4.85s/it]


,question,deterministic_generation
0,What is George Rankin's occupation?,What is George Rankin's occupation?\n\nGeorge ...
1,What is John Mayne's occupation?,What is John Mayne's occupation?\n\nJohn Mayne...
2,What is Henry Feilden's occupation?,What is Henry Feilden's occupation?\n\nHenry F...
3,What is Kathy Saltzman's occupation?,What is Kathy Saltzman's occupation?\n\nKathy ...
4,What is Eleanor Davis's occupation?,What is Eleanor Davis's occupation?\n\nEleanor...


In [ ]:
answer_df

,question,deterministic_generation
0,What is George Rankin's occupation?,What is George Rankin's occupation?\n\nGeorge ...
1,What is John Mayne's occupation?,What is John Mayne's occupation?\n\nJohn Mayne...
2,What is Henry Feilden's occupation?,What is Henry Feilden's occupation?\n\nHenry F...
3,What is Kathy Saltzman's occupation?,What is Kathy Saltzman's occupation?\n\nKathy ...
4,What is Eleanor Davis's occupation?,What is Eleanor Davis's occupation?\n\nEleanor...


In [ ]:
# @title Generate sample answers for questions on KEEN PopQA

# Grab random 5 questions from the dataset and generate deterministic answer and display

sample = keen_popqa_dataset["train"].select(range(5))
sample_df = sample.to_pandas()

# Grab question and possible_answers field and display
sample_df = sample_df[["question", "possible_answers", "label"]]
sample_df.head()

,question,possible_answers,label
0,Who was the composer of '71?,['David Holmes'],head
1,What genre is '71?,"['crime film', 'thriller movie', 'film action'...",head
2,What is the country of origin of '71?,"['GBR', 'The UK', 'The United Kingdom of Great...",head
3,Who was the director of '71?,['Yann Demange'],head
4,What color is '71?,"['colour film', 'color film', 'color', 'colour...",head


In [ ]:
# Let' use generate function to generate answer to a question

# Check for whether is it cuda or cpu
# params to generate function(model, tokenizer, device, df, hs_patch_config=None)
device = "cuda" if torch.cuda.is_available() else "cpu"
gpt_model.model.to(device) # Move the model to the correct device
keen_answer_df = generate(gpt_model.model, gpt_model.tokenizer, device, sample_df)
keen_answer_df.head()

5it [00:24,  4.88s/it]


,question,deterministic_generation
0,Who was the composer of '71?,Who was the composer of '71?\n\nI don't know. ...
1,What genre is '71?,"What genre is '71?\n\n""It's a very specific ge..."
2,What is the country of origin of '71?,What is the country of origin of '71?\n\nThe c...
3,Who was the director of '71?,Who was the director of '71?\n\nI was the dire...
4,What color is '71?,What color is '71?\n\nThe color of the '71 is ...


In [ ]:
# @title Let us experiment with pushing this sample to HF

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(keen_answer_df, preserve_index=False)
dataset_dict = DatasetDict({"train": ds})

In [ ]:
repo_id = "kokolamba/keen_popqa_gpt2xl_generations"

dataset_dict.push_to_hub(repo_id, private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########| 2.78kB / 2.78kB            

CommitInfo(commit_url='https://huggingface.co/datasets/kokolamba/keen_popqa_gpt2xl_generations/commit/a80e839803f9aea64db0c4452d6629c73a0fffcb', commit_message='Upload dataset', commit_description='', oid='a80e839803f9aea64db0c4452d6629c73a0fffcb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kokolamba/keen_popqa_gpt2xl_generations', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kokolamba/keen_popqa_gpt2xl_generations'), pr_revision=None, pr_num=None)

In [ ]:
# @title Load and generate completions for to all KEEN PopQA questions

keen_popqa_dataset = load_dataset("dhgottesman/keen_estimating_knowledge_in_llms", data_files="popqa_questions.csv")
len(keen_popqa_dataset["train"])

In [ ]:
# Enter the HF token

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Start with gpt2 for testing that code works
# Subsequently used gpt2-xl

gpt_model = GPTModelAndTokenizer(model_name="gpt2-xl")

In [ ]:
# Let's use generate function to generate answer to a question

# Check for whether is it cuda or cpu
# params to generate function(model, tokenizer, device, df, hs_patch_config=None)
device = "cuda" if torch.cuda.is_available() else "cpu"
gpt_model.model.to(device) # Move the model to the correct device
keen_answer_df = generate(gpt_model.model, gpt_model.tokenizer, device, keen_popqa_dataset)

In [ ]:
len(keen_answer_df)

In [ ]:
keen_answer_df.head()

In [ ]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(keen_answer_df, preserve_index=False)
dataset_dict = DatasetDict({"train": ds})

In [ ]:
repo_id = "kokolamba/keen_popqa_gpt2xl_generations"

dataset_dict.push_to_hub(repo_id, private=False)

In [ ]:
# import datasets from repo_id
from datasets import load_dataset

repo_id = "kokolamba/keen_popqa_gpt2xl_generations"
dataset = load_dataset(repo_id)
dataset = dataset["train"].to_pandas()
dataset.head()

,question,deterministic_generation
0,Who was the composer of '71?,Who was the composer of '71?\n\nI don't know. ...
1,What genre is '71?,"What genre is '71?\n\n""It's a very specific ge..."
2,What is the country of origin of '71?,What is the country of origin of '71?\n\nThe c...
3,Who was the director of '71?,Who was the director of '71?\n\nI was the dire...
4,What color is '71?,What color is '71?\n\nThe color of the '71 is ...
